In [48]:
import pandas as pd
import numpy as np
import re
import string
import unidecode
import nltk
import os


# Aqui os .txt estao dentro da pasta loop
files = os.listdir('loop') # mudar para o diretorio que tenha todos os .txt dos tweets
lista_pandas = list()
regex = re.compile(r'(?<=tweets_)[a-zA-Z\s]{2,}')

for file in files:
    d = pd.read_csv(os.path.join('loop',file), sep="\t")
    d['tag'] = regex.search(file).group()
    lista_pandas.append(d)
    print(file)

data = pd.concat(lista_pandas)
data.reset_index(drop=True, inplace=True)
print(data.shape)
# se gerar erro, abrir o .txt no notepad e salvar em UTF-8

tweets_banco do brasil_2018_18_04+.txt
tweets_banco do brasil_2018_18_04.txt
tweets_banco do brasil_2018_18_04_.txt
tweets_BancoInter_2018_18_04.txt
tweets_BancoInter_2018_18_04_.txt
tweets_BancoInter_2018_19_04+.txt
tweets_bradesco_2018_18_04+.txt
tweets_bradesco_2018_18_04.txt
tweets_bradesco_2018_18_04_.txt
tweets_itau_2018_18_04+.txt
tweets_itau_2018_18_04.txt
tweets_itau_2018_18_04_.txt
tweets_nubank_2018_18_04.txt
tweets_nubank_2018_18_04_.txt
tweets_nubank_2018_19_04+.txt
tweets_santander_2018_18_04+.txt
tweets_santander_2018_18_04.txt
tweets_santander_2018_18_04_.txt
(58061, 11)


In [49]:
data['conteudo'].head(10)

0    RT @Chanyyuta: Peço a ajuda  para salvar a  vi...
1    RT @Chanyyuta: Peço a ajuda  para salvar a  vi...
2    RT @Chanyyuta: Peço a ajuda  para salvar a  vi...
3    Lembrar de nunca mais usar o débito em conta o...
4    RT @Chanyyuta: Peço a ajuda  para salvar a  vi...
5    RT @Chanyyuta: Peço a ajuda  para salvar a  vi...
6    Eu tinha quatro centavos na minha conta no ban...
7    Olha a inflação de 1.75 do governo .\r\n\r\nFo...
8    Eu odeio tanto o Banco do Brasil q vcs não ima...
9    RT @ramosbrunow: terceira temporada de La Casa...
Name: conteudo, dtype: object

In [50]:
# Criando uma coluna com as contas citadas no campo conteudo @...
regex = re.compile(r'@([^\s]+)') 
data['citados'] = data['conteudo'].apply(lambda x: regex.findall(str(x)))
data['n_citados'] = data['citados'].apply(lambda x: len(x))
data['citados'] = data['citados'].apply(lambda x: ' '.join(x) if isinstance(x, list) else np.NaN)
data['citados'] = data['citados'].apply(lambda x: x.strip(string.punctuation))
data['citados'] = data['citados'].apply(lambda x: x.lower())
data.groupby('tag').data.count()

tag
BancoInter          4788
banco do brasil    11800
bradesco            9558
itau               11801
nubank              9134
santander          10976
Name: data, dtype: int64

In [70]:
interacao = data[data['citados'].str.contains('itau')]
interacao = interacao[interacao.tag!='itau']
interacao['conteudo'] = interacao['conteudo'].str.replace('\t', '')
interacao['conteudo'] = interacao['conteudo'].str.replace('\n', ' ')
interacao.drop_duplicates(subset='conteudo', inplace=True)

In [55]:
for a in interacao['conteudo']:
    print(a)  

Por favor, pior Banco do Brasil @itau , façam o favor de melhorarem o atendimento na rede de agências nem a Caixa econômica Federal demora tanto quanto vcs!
@itau Eu sei que é possível, sempre fiz, mas de um tempo pra cá não rola. Aparece uma mensagem dizendo "não foi possível realizar, tente novamente mais tarde". Já tentei pro Banco do brasil e santander :(
@itau O Bradesco já tem, o banco do Brasil também e a Caixa... tem também o Santander, o Banrisul e o banco Inter.E até o banco Neon! Mas cliente Itaú não tem...
@itau o serviço de vocês e muito porco, queria fazer uma transferência pelo app para o banco do Brasil e vocês não dão a opção de por o digito da agência, e pelo que vi no reclame aqui essa merda já tem mais de 3 anos e vocês ainda não arrumaram, estão de parabéns
@itau Mas o ti de vocês é pior do que o banco Neon, do banco inter, do banco do Brasil, caixa, Santander e Banrisul? Pois todos eles oferecem experiência de alta qualidade com esse serviço
@JuMagalhaesBH @itau V

In [71]:
print(interacao.shape)
interacao.apply(pd.Series.nunique, axis=0)

(232, 13)


Unnamed: 0    232
query           5
data          232
usuario       167
conteudo      232
retweets        4
favoritos       9
local          12
seguidores    159
amigos        154
tag             5
citados       143
n_citados      12
dtype: int64

In [57]:
produto_marca = data[(data.tag=='itau') & (data.usuario!='Itaú')] # nao tenho interesse no post do usuario
produto_marca.drop_duplicates(subset='conteudo', inplace=True)
produto_marca['conteudo'] = produto_marca['conteudo'].str.replace('\t', '')
for a in produto_marca['conteudo']:
    print(a)

C:\Users\VictorVT\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)
C:\Users\VictorVT\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


@Bradesco Sou correntista de do @Bradesco por conveniência do município. Pela minha experiência o melhor banco privado é o @itau .
RT @DopeKstore: ?CEG: Love Yourself TEAR?

?R$115,00 - Unidade 
?R$410,00 - Combo
?Pgto - Depósito ou Transferência bancária no Itau ou Cai…
RT @RAgapto: De repente falta energia na agência do Itaú de Manaus. Quando os eletricistas vão até à central de energia, descobrem a causa…
RT @DopeKstore: ?PROMOÇÃO?
?R$35,00 + frete cada
?Reserva até 12/04
?Pgto Itaú ou Caixa Econômica 
?Pedir na DM com Nome completo e email h…
@itau @Itaucard vocês deveriam tomar providências sobre esse perfil que está fazendo propaganda nas redes sociais... https://t.co/SERBzyngSB
@jccn83 enquanto isso a Itaú corretora mandando seus clientes venderem seus bova's
esqueci que tenho que ir no itaú amanhã aaaa :(
@itau CHAMA NO PROBLEMINHA (de) preciso de uma ajuda
@itau Boa pergunta. Ninguém disse, em nenhuma das vezes que eu atendi.

Outras boas perguntas: e se o telefone não fosse m

In [58]:
produto_marca['n_citados'].value_counts()

0     2187
1     1878
2      501
3      151
50     139
4       59
5       21
11      17
20      16
6       15
10      13
7        7
12       3
8        2
9        2
21       1
15       1
14       1
22       1
51       1
Name: n_citados, dtype: int64

In [60]:
produto_marca = produto_marca[produto_marca.n_citados < 6]
produto_marca.shape

(4797, 13)

In [73]:
interacao_um = interacao.iloc[:130,:]
interacao_dois = interacao.iloc[130:,:]

interacao_um.to_csv("interacao_parte1_1904.txt", sep="\t")
interacao_dois.to_csv("interacao_parte2_1904.txt", sep="\t")

In [77]:
produto_marca_um = produto_marca.iloc[:200,:]
produto_marca_dois = produto_marca.iloc[200:400,:]
produto_marca_tres = produto_marca.iloc[401:601,:]
produto_marca_quatro = produto_marca.iloc[602:802,:]


produto_marca_um.to_csv("produto_marca_parte_um_1904.txt", sep="\t")
produto_marca_dois.to_csv("produto_marca_parte_dois_1904.txt", sep="\t")
produto_marca_tres.to_csv("produto_marca_parte_tres_1904.txt", sep="\t")
produto_marca_quatro.to_csv("produto_marca_parte_quatro_1904.txt", sep="\t")